In [1]:
import geopandas as gpd
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from pathlib import Path
import sys
import ribasim

# add directory with ribasim_lumping-repository
from ribasim_lumping.ribasim_model_results.ribasim_results import plot_results_basin_ribasim_model, plot_results_basins_ribasim_model

c:\Users\NLHARN\Documents\PROGRAMMING\ribasim_lumping_venv\Lib\site-packages\dask\dataframe\__init__.py:31: FutureWarning: 
Dask dataframe query planning is disabled because dask-expr is not installed.

You can install it with `pip install dask[dataframe]` or `conda install dask`.
This will raise in a future version.

  warnings.warn(msg, FutureWarning)
c:\Users\NLHARN\Documents\PROGRAMMING\ribasim_lumping_venv\Lib\site-packages\dask\dataframe\__init__.py:31: FutureWarning: 
Dask dataframe query planning is disabled because dask-expr is not installed.

You can install it with `pip install dask[dataframe]` or `conda install dask`.
This will raise in a future version.

  warnings.warn(msg, FutureWarning)
c:\Users\NLHARN\Documents\PROGRAMMING\ribasim_lumping_venv\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
c:\Use

In [2]:
import shutil
from pathlib import Path

import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
from ribasim import Allocation, Model, Node
from ribasim.nodes import (
    basin,
    discrete_control,
    flow_boundary,
    fractional_flow,
    level_boundary,
    level_demand,
    linear_resistance,
    manning_resistance,
    outlet,
    pid_control,
    pump,
    tabulated_rating_curve,
    user_demand,
)
from shapely.geometry import Point

In [3]:
datadir = Path("data")
shutil.rmtree(datadir, ignore_errors=True)

In [4]:
model = Model(starttime="2020-01-01", endtime="2021-01-01", crs="EPSG:4326")

In [5]:
time = pd.date_range(model.starttime, model.endtime)
day_of_year = time.day_of_year.to_numpy()
seconds_per_day = 24 * 60 * 60
evaporation = (
    (-1.0 * np.cos(day_of_year / 365.0 * 2 * np.pi) + 1.0) * 0.0025 / seconds_per_day
)
rng = np.random.default_rng(seed=0)
precipitation = (
    rng.lognormal(mean=-1.0, sigma=1.7, size=time.size) * 0.001 / seconds_per_day
)

# Convert steady forcing to m/s
# 2 mm/d precipitation, 1 mm/d evaporation

basin_data = [
    basin.Profile(area=[0.01, 1000.0], level=[0.0, 1.0]),
    basin.Time(
        time=pd.date_range(model.starttime, model.endtime),
        drainage=0.0,
        potential_evaporation=evaporation,
        infiltration=0.0,
        precipitation=precipitation,
        urban_runoff=0.0,
    ),
    basin.State(level=[1.4]),
]

model.basin.add(Node(1, Point(0.0, 0.0)), basin_data)
model.basin.add(Node(3, Point(2.0, 0.0)), basin_data)
model.basin.add(Node(6, Point(3.0, 2.0)), basin_data)
model.basin.add(Node(9, Point(5.0, 0.0)), basin_data)

In [ ]:
profile = pd.DataFrame(
    data={
        "node_id": [1, 1, 3, 3, 6, 6, 9, 9],
        "area": [0.01, 1000.0] * 4,
        "level": [0.0, 1.0] * 4,
    }
)

# Convert steady forcing to m/s
# 2 mm/d precipitation, 1 mm/d evaporation
seconds_in_day = 24 * 3600
precipitation = 0.002 / seconds_in_day
evaporation = 0.001 / seconds_in_day

static = pd.DataFrame(
    data={
        "node_id": [0],
        "drainage": [0.0],
        "potential_evaporation": [evaporation],
        "infiltration": [0.0],
        "precipitation": [precipitation],
        "urban_runoff": [0.0],
    }
)
static = static.iloc[[0, 0, 0, 0]]
static["node_id"] = [1, 3, 6, 9]

basin = ribasim.Basin(profile=profile, static=static)

In [ ]:
basin.static

In [ ]:
basin.profile

In [ ]:
linear_resistance = ribasim.LinearResistance(
    static=pd.DataFrame(
        data={"node_id": [10, 12], "resistance": [5e3, (3600.0 * 24) / 100.0]}
    )
)
linear_resistance.static

In [ ]:
manning_resistance = ribasim.ManningResistance(
    static=pd.DataFrame(
        data={
            "node_id": [2],
            "length": [900.0],
            "manning_n": [0.04],
            "profile_width": [6.0],
            "profile_slope": [3.0],
        }
    )
)
manning_resistance.static

In [ ]:
# Discharge: lose 1% of storage volume per day at storage = 1000.0.
q1000 = 1000.0 * 0.01 / seconds_in_day

rating_curve = ribasim.TabulatedRatingCurve(
    static=pd.DataFrame(
        data={
            "node_id": [4, 4],
            "level": [0.0, 1.0],
            "discharge": [0.0, q1000],
        }
    )
)
rating_curve.static

In [ ]:
fractional_flow = ribasim.FractionalFlow(
    static=pd.DataFrame(
        data={
            "node_id": [5, 8, 13],
            "fraction": [0.3, 0.6, 0.1],
        }
    )
)
fractional_flow.static

In [ ]:
pump = ribasim.Pump(
    static=pd.DataFrame(
        data={
            "node_id": [7],
            "flow_rate": [0.5 / 3600],
        }
    )
)
pump.static

In [ ]:
level_boundary = ribasim.LevelBoundary(
    static=pd.DataFrame(
        data={
            "node_id": [11, 17],
            "level": [0.5, 1.5],
        }
    )
)

In [ ]:
level_boundary.static

In [ ]:
level_boundary.time

In [ ]:
flow_boundary = ribasim.FlowBoundary(
    static=pd.DataFrame(
        data={
            "node_id": [15, 16],
            "flow_rate": [1e-4, 1e-4],
        }
    )
)
flow_boundary.static

In [ ]:
terminal = ribasim.Terminal(
    static=pd.DataFrame(
        data={
            "node_id": [14],
        }
    )
)
terminal.static

In [ ]:
xy = np.array(
    [
        (0.0, 0.0),  # 1: Basin,
        (1.0, 0.0),  # 2: ManningResistance
        (2.0, 0.0),  # 3: Basin
        (3.0, 0.0),  # 4: TabulatedRatingCurve
        (3.0, 1.0),  # 5: FractionalFlow
        (3.0, 2.0),  # 6: Basin
        (4.0, 1.0),  # 7: Pump
        (4.0, 0.0),  # 8: FractionalFlow
        (5.0, 0.0),  # 9: Basin
        (6.0, 0.0),  # 10: LinearResistance
        (2.0, 2.0),  # 11: LevelBoundary
        (2.0, 1.0),  # 12: LinearResistance
        (3.0, -1.0),  # 13: FractionalFlow
        (3.0, -2.0),  # 14: Terminal
        (3.0, 3.0),  # 15: FlowBoundary
        (0.0, 1.0),  # 16: FlowBoundary
        (6.0, 1.0),  # 17: LevelBoundary
    ]
)
node_xy = gpd.points_from_xy(x=xy[:, 0], y=xy[:, 1])

node_id, node_type = ribasim.Node.get_node_ids_and_types(
    basin,
    manning_resistance,
    rating_curve,
    pump,
    fractional_flow,
    linear_resistance,
    level_boundary,
    flow_boundary,
    terminal,
)

# Make sure the feature id starts at 1: explicitly give an index.
node = ribasim.Node(
    static=gpd.GeoDataFrame(
        data={"type": node_type},
        index=pd.Index(node_id, name="fid"),
        geometry=node_xy,
        crs="EPSG:28992",
    )
)
node.static

In [ ]:
from_id = np.array(
    [1, 2, 3, 4, 4, 5, 6, 8, 7, 9, 11, 12, 4, 13, 15, 16, 10], dtype=np.int64
)
to_id = np.array(
    [2, 3, 4, 5, 8, 6, 7, 9, 9, 10, 12, 3, 13, 14, 6, 1, 17], dtype=np.int64
)
lines = ribasim.utils.geometry_from_connectivity(node, from_id, to_id)
edge = ribasim.Edge(
    static=gpd.GeoDataFrame(
        data={
            "from_node_id": from_id,
            "to_node_id": to_id,
            "edge_type": len(from_id) * ["flow"],
        },
        geometry=lines,
        crs="EPSG:28992",
    )
)
lines

In [ ]:
model = ribasim.Model(
    modelname="basic",
    node=node,
    edge=edge,
    basin=basin,
    level_boundary=level_boundary,
    flow_boundary=flow_boundary,
    pump=pump,
    linear_resistance=linear_resistance,
    manning_resistance=manning_resistance,
    tabulated_rating_curve=rating_curve,
    fractional_flow=fractional_flow,
    terminal=terminal,
    starttime="2020-01-01 00:00:00",
    endtime="2021-01-01 00:00:00",
)

In [ ]:
model.plot()

In [ ]:
node.static

In [ ]:
results_dir = "..\\..\\..\\ribasim_lumping_data\\examples"
simulation_code = "basic_model_with_static_forcing"
simulation_path = Path(results_dir, simulation_code)

In [ ]:
model.solver = {"abstol": 1e-10, "reltol": 1e-8}
model.write(simulation_path)
bat_file = Path(simulation_path, "run_ribasim.bat")
with open(bat_file, "w") as f:
    f.write("..\\..\\ribasim_cli\\bin\\ribasim.exe ribasim.toml\n")

In [ ]:
plot_results_basins_ribasim_model(
    ribasim_model=model,
    simulation_path=simulation_path,
)